In [ ]:
import pandas as pd
import numpy as np

# computation methods for decision tree
# set of instances represented by pandas dataframe 
# (makes computations and groupbys over set easy)

# compute entropy of instances / class_val
def entropy(df, class_val):
    freqs = df[class_val].value_counts(normalize=True)
    return -(freqs * np.log2(freqs)).sum()

# compute gain of a instances / class_val over attribute
def gain(df, class_val, attribute, current_entropy):
    weighted_entropy = 0
    for attr_val, df_split in df.groupby([attribute]):
        weight = df_split.shape[0] / df.shape[0]
        split_entropy = entropy(df_split, class_val)
        weighted_entropy += weight * split_entropy
        print('*****', attribute, attr_val, weight, split_entropy)
    print('****', attribute, 'weighted_entropy', weighted_entropy)
    return current_entropy - weighted_entropy
        
# determine best split attribute of instances / class_val
def find_split_attr(prefix, df, class_val):

    (best_attr, best_gain) = (None, -1)
    base_entropy = entropy(df, class_val)
    print('*** base_entropy', prefix, base_entropy)
    
    # dirty check for zero
    if (base_entropy < 0.000001):
        return None

    # dirty way to ignore attributes we've inspected
    ignore_attrs = [x for x,_ in prefix]
    ignore_attrs.append(class_val)
    
    for attr in list(df.columns):
        if attr in (ignore_attrs):
            continue
        print('**** evaluate', prefix, attr)
        this_gain = gain(df, class_val, attr, base_entropy)
        print('****', attr, 'gain', this_gain)
        if (this_gain > best_gain):
            (best_attr, best_gain) = (attr, this_gain)
            
    if best_attr is None:
        return None
    
    print('*** choose', best_attr, best_gain)
    # return (best_attr, best_gain, [[prefix, split_df], ...])
    return (best_attr, best_gain, [[prefix + [(best_attr, split_val)], split_df] for (split_val, split_df) in df.groupby([best_attr])])


In [ ]:
# import CSV
# iteratively split the instances on entropy-minimizing attributes

df = pd.read_csv('./hw2-data-data.csv')
class_val = 'HOUSEPET'

# initial split evaluation to seed stack of nodes for evaluation
tree = []
prefix = []
print('examine', prefix)
result = find_split_attr(prefix, df, class_val)
splits = result[2]
tree.append(([], result[0], ('gain', result[1])))

# while nodes remaining to split
while splits:
    (prefix, df_split) = splits.pop()
    print('examine', prefix)
    result = find_split_attr(prefix, df_split, class_val)
    if result is None:
        class_val_counts = df_split[class_val].value_counts()
        tree.append((prefix, ('count', class_val_counts.index[0], class_val_counts[0])))
    else:
        tree.append((prefix, result[0], ('gain', result[1])))
        splits.extend(result[2])

# dump the tree
tree.sort()
tree

In [ ]:
def gini(s):
    g = 1
    for i in s:
        g -= ((i)/ sum(s))**2
    return g

def weighted_gini(s1, s2):
    print('***', s1, s2)
    w1 = sum(s1) / (sum(s1) + sum(s2))
    w2 = sum(s2) / (sum(s1) + sum(s2))
    g1 = gini(s1)
    g2 = gini(s2)
    print(f'*** weight1: {w1} gini1: {g1} weight2: {w2} gini2: {g2}')
    return ((w1 * g1) + (w2 * g2))

def weighted_gini_partition(attr, part_vals):
    value_counts1 = df[df[attr].isin(part_vals)][class_val].value_counts()
    value_counts2 = df[~df[attr].isin(part_vals)][class_val].value_counts()
    return (weighted_gini(value_counts1.tolist(), value_counts2.tolist()))


attr = 'SHOTS'
parts = ['all', 'some', 'none']
for part in parts:
    print(f'\nshots == {part}')
    print ('gini:', weighted_gini_partition(attr, [part]))
